In [1]:

import pandas as pd
df = pd.read_csv("/kaggle/input/System-Threat-Forecaster/train.csv")
test_data = pd.read_csv("/kaggle/input/System-Threat-Forecaster/test.csv")
sample_submission = pd.read_csv('/kaggle/input/System-Threat-Forecaster/sample_submission.csv')
df.head()


,MachineID,ProductName,EngineVersion,AppVersion,SignatureVersion,IsBetaUser,RealTimeProtectionState,IsPassiveModeEnabled,AntivirusConfigID,NumAntivirusProductsInstalled,...,IsSecureBootEnabled,IsVirtualDevice,IsTouchEnabled,IsPenCapable,IsAlwaysOnAlwaysConnectedCapable,IsGamer,RegionIdentifier,DateAS,DateOS,target
0,f541bae429089117c4aac39c90dd3416,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1003.0,0,7.0,0,53447.0,1.0,...,0,0.0,1,0,1.0,0.0,6.0,2018-09-10 10:11:00,2018-04-17,0
1,dc2b14d9ce3a0ce4050bb640190f2ca5,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1465.0,0,7.0,0,53447.0,1.0,...,1,0.0,0,0,0.0,0.0,10.0,2018-08-16 00:01:00,2018-08-14,1
2,fd20c5f010e9c5f91ad1c6b3e0da68a0,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1546.0,0,7.0,0,53447.0,1.0,...,0,0.0,0,0,0.0,1.0,6.0,2018-09-20 23:20:00,2018-09-11,1
3,38711eae85eb77a72ec5dfdf27eb2a76,win8defender,1.1.15200.1,4.12.17007.18011,1.275.1141.0,0,7.0,0,46413.0,2.0,...,1,0.0,0,0,0.0,0.0,12.0,2018-09-14 00:32:00,2018-01-03,1
4,32607c9a543a9214e2c7e45800ed4849,win8defender,1.1.15200.1,4.13.17134.228,1.275.1283.0,0,7.0,0,40466.0,2.0,...,0,0.0,0,0,0.0,1.0,7.0,2018-09-15 19:34:00,2018-09-11,0


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 76 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   MachineID                           100000 non-null  object 
 1   ProductName                         100000 non-null  object 
 2   EngineVersion                       100000 non-null  object 
 3   AppVersion                          100000 non-null  object 
 4   SignatureVersion                    100000 non-null  object 
 5   IsBetaUser                          100000 non-null  int64  
 6   RealTimeProtectionState             99934 non-null   float64
 7   IsPassiveModeEnabled                100000 non-null  int64  
 8   AntivirusConfigID                   99924 non-null   float64
 9   NumAntivirusProductsInstalled       99924 non-null   float64
 10  NumAntivirusProductsEnabled         99924 non-null   float64
 11  HasTpm                     

In [3]:
from sklearn.impute import SimpleImputer

# Identify numerical and categorical columns
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
cat_cols = df.select_dtypes(include=['object']).columns
num_cols = num_cols.drop('target', errors='ignore')

# Impute numerical columns with mean
imputer_num = SimpleImputer(strategy='mean')
df[num_cols] = imputer_num.fit_transform(df[num_cols])
test_data[num_cols] = imputer_num.transform(test_data[num_cols])

# Impute categorical columns with most frequent value
imputer_cat = SimpleImputer(strategy='most_frequent')
df[cat_cols] = imputer_cat.fit_transform(df[cat_cols])
test_data[cat_cols] = imputer_cat.transform(test_data[cat_cols])

print("Missing values handled successfully!")

Missing values handled successfully!


In [4]:
df['DateAS'] = pd.to_datetime(df['DateAS']).dt.month
df['DateOS'] = pd.to_datetime(df['DateOS']).dt.month
test_data['DateAS'] = pd.to_datetime(test_data['DateAS']).dt.month
test_data['DateOS'] = pd.to_datetime(test_data['DateOS']).dt.month

In [5]:
cardinality = df.nunique().sort_values(ascending=False)

low_cardinality = cardinality[cardinality == 1].index.tolist()
df = df.drop(columns=low_cardinality)
test_data = test_data.drop(columns=low_cardinality)
df.shape

(100000, 73)

In [6]:
categorical_cols = df.select_dtypes(include=['object']).columns

selected_features = [col for col in categorical_cols if df[col].nunique() <= 10]
high_cardinality_features = [col for col in categorical_cols if df[col].nunique() > 10]
df = pd.get_dummies(df, columns=selected_features, drop_first=True)
test_data= pd.get_dummies(test_data, columns=selected_features, drop_first=True)
df

,MachineID,EngineVersion,AppVersion,SignatureVersion,RealTimeProtectionState,IsPassiveModeEnabled,AntivirusConfigID,NumAntivirusProductsInstalled,NumAntivirusProductsEnabled,HasTpm,...,LicenseActivationChannel_Retail,LicenseActivationChannel_Retail:TB:Eval,LicenseActivationChannel_Volume:GVLK,LicenseActivationChannel_Volume:MAK,FlightRing_NOT_SET,FlightRing_RP,FlightRing_Retail,FlightRing_Unknown,FlightRing_WIF,FlightRing_WIS
0,f541bae429089117c4aac39c90dd3416,1.1.15200.1,4.18.1807.18075,1.275.1003.0,7.0,0.0,53447.0,1.0,1.0,1.0,...,True,False,False,False,False,False,True,False,False,False
1,dc2b14d9ce3a0ce4050bb640190f2ca5,1.1.15100.1,4.18.1807.18075,1.273.1465.0,7.0,0.0,53447.0,1.0,1.0,1.0,...,False,False,False,False,False,False,True,False,False,False
2,fd20c5f010e9c5f91ad1c6b3e0da68a0,1.1.15200.1,4.18.1807.18075,1.275.1546.0,7.0,0.0,53447.0,1.0,1.0,1.0,...,False,False,False,False,False,False,True,False,False,False
3,38711eae85eb77a72ec5dfdf27eb2a76,1.1.15200.1,4.12.17007.18011,1.275.1141.0,7.0,0.0,46413.0,2.0,1.0,1.0,...,False,False,False,False,False,False,True,False,False,False
4,32607c9a543a9214e2c7e45800ed4849,1.1.15200.1,4.13.17134.228,1.275.1283.0,7.0,0.0,40466.0,2.0,1.0,1.0,...,True,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,a378b2196a7a1d8ee0d4f989ee46730e,1.1.15200.1,4.18.1807.18075,1.275.1582.0,7.0,0.0,68585.0,2.0,1.0,1.0,...,True,False,False,False,False,False,True,False,False,False
99996,7f4433ca05d24b9bbdd3932f8b05e61f,1.1.15100.1,4.18.1806.18062,1.273.1156.0,7.0,0.0,11280.0,2.0,1.0,1.0,...,True,False,False,False,False,False,True,False,False,False
99997,d6852dfff534e03d5ddfd3c7875ab072,1.1.15200.1,4.18.1807.18075,1.275.209.0,7.0,0.0,53447.0,1.0,1.0,1.0,...,False,False,False,False,False,False,True,False,False,False
99998,70a8dc682128c9ff78a3de5aea9dac74,1.1.14901.4,4.16.17656.18052,1.269.641.0,7.0,0.0,53447.0,1.0,1.0,1.0,...,False,False,False,False,False,False,False,True,False,False


In [7]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
for col in high_cardinality_features:
    # Apply Label Encoding
    df[col] = encoder.fit_transform(df[col])
    test_data[col] = encoder.fit_transform(test_data[col]) 

# Display the transformed dataset
df.head()

,MachineID,EngineVersion,AppVersion,SignatureVersion,RealTimeProtectionState,IsPassiveModeEnabled,AntivirusConfigID,NumAntivirusProductsInstalled,NumAntivirusProductsEnabled,HasTpm,...,LicenseActivationChannel_Retail,LicenseActivationChannel_Retail:TB:Eval,LicenseActivationChannel_Volume:GVLK,LicenseActivationChannel_Volume:MAK,FlightRing_NOT_SET,FlightRing_RP,FlightRing_Retail,FlightRing_Unknown,FlightRing_WIF,FlightRing_WIS
0,95607,34,31,2218,7.0,0.0,53447.0,1.0,1.0,1.0,...,True,False,False,False,False,False,True,False,False,False
1,85800,33,31,1906,7.0,0.0,53447.0,1.0,1.0,1.0,...,False,False,False,False,False,False,True,False,False,False
2,98689,34,31,2331,7.0,0.0,53447.0,1.0,1.0,1.0,...,False,False,False,False,False,False,True,False,False,False
3,21877,34,17,2253,7.0,0.0,46413.0,2.0,1.0,1.0,...,False,False,False,False,False,False,True,False,False,False
4,19567,34,22,2265,7.0,0.0,40466.0,2.0,1.0,1.0,...,True,False,False,False,False,False,True,False,False,False


In [8]:
num_cols = df.select_dtypes(include=['int64','int32','float64']).columns.drop('target', errors='ignore')
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])
test_data[num_cols] = scaler.fit_transform(test_data[num_cols]) 

In [9]:
df= df.drop(columns=list(set(df.columns)-(set(test_data.columns)|{'target'})), axis=1)

In [10]:
from sklearn.preprocessing import LabelEncoder

# Automatically identify categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns
test_categorical_cols = test_data.select_dtypes(include=['object']).columns

# Apply Label Encoding
encoder = LabelEncoder()
for col in categorical_cols:
    df[col] = encoder.fit_transform(df[col])
for col in test_categorical_cols:
    test_data[col] = encoder.fit_transform(test_data[col]) 

In [11]:
from sklearn.model_selection import train_test_split
X = df.drop(columns=['target'])   # Features
y = df['target']     # Target column
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Initialize the XGBoost classifier
xgb = XGBClassifier(n_estimators=100, random_state=42)

# Train the model
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_xgb)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.6230


In [13]:
test_data = test_data[X.columns]

In [14]:
test_predictions = xgb.predict(test_data)  # Generate predictions

# Create submission DataFrame (Ensure sample_submission exists)
submission = sample_submission.copy()
submission['target'] = test_predictions  

# If classification-like labels are needed (Assuming binary case)
# Ensure model output is properly rounded or thresholded
submission['target'] = submission['target'].apply(lambda x: '1' if x > 0.5 else '0')

# Save to CSV
submission.to_csv('submission.csv', index=False)

# Output file ready for submission
print("Submission file created.")

Submission file created.


In [15]:
# Import necessary libraries
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Assuming 'X' is the feature matrix and 'y' is the target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define Logistic Regression model
log_reg = LogisticRegression()

# Define hyperparameter grid
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Regularization strength
    'solver': ['lbfgs', 'liblinear']  # Optimizers
}

# Perform GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(log_reg, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# Best model evaluation
best_log_reg = grid_search.best_estimator_
y_pred = best_log_reg.predict(X_test_scaled)

# Print results
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)
print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Best Parameters: {'C': 10, 'solver': 'liblinear'}
Best Cross-Validation Accuracy: 0.6014375000000001
Test Accuracy: 0.6043
Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.55      0.58      9895
           1       0.60      0.66      0.63     10105

    accuracy                           0.60     20000
   macro avg       0.61      0.60      0.60     20000
weighted avg       0.61      0.60      0.60     20000

